### In a nutshell:
- Regulatory Build elements will be intersected with (A) variants and (B) TSS to obtain genomic ranges.
- A and B ranges will be intersected with the different datasets.
- A-B contact will be calculated from HiC

In [1]:
import pandas as pd
import numpy as np
import pybedtools
import gffpandas.gffpandas as gffpd

# Reg build elements

In [2]:
regbuild_file='/nfs/research1/zerbino/jhidalgo/inteql_GTEX_v8/data/original-data/RegBuild/homo_sapiens.GRCh38.CD14_monocyte_1.Regulatory_Build.regulatory_activity.20190329.gff'

In [3]:
regbuild=gffpd.read_gff3(regbuild_file).attributes_to_columns()[['seq_id','bound_start','bound_end','regulatory_feature_stable_id','type','activity']]

/homes/jhidalgo/.local/lib/python3.8/site-packages/gffpandas/gffpandas.py:6: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  return Gff3DataFrame(input_file)


In [4]:
regbuild['seq_id']='chr'+regbuild['seq_id'].astype(str)

In [5]:
regbuild=regbuild[regbuild['seq_id'] == 'chr1']

In [6]:
regbuild=regbuild.iloc[:,0:4].merge(regbuild[['regulatory_feature_stable_id','type','activity']].replace({'REPRESSED':1,'INACTIVE':2,'POISED':3,'ACTIVE':4,'.':0}).pivot_table(values=['activity'], index=['regulatory_feature_stable_id'],columns='type').droplevel(0,axis=1),how='left',left_on='regulatory_feature_stable_id',right_index=True)

In [7]:
regbuild

,seq_id,bound_start,bound_end,regulatory_feature_stable_id,CTCF_binding_site,TF_binding_site,enhancer,open_chromatin_region,promoter,promoter_flanking_region
20264,chr1,100034201,100035200,ENSR00000253249,4.0,NaN,NaN,NaN,NaN,NaN
20265,chr1,100127601,100127800,ENSR00000366775,2.0,NaN,NaN,NaN,NaN,NaN
20266,chr1,100130401,100130600,ENSR00000928323,2.0,NaN,NaN,NaN,NaN,NaN
20267,chr1,100135001,100135200,ENSR00000366777,2.0,NaN,NaN,NaN,NaN,NaN
20268,chr1,100147801,100148400,ENSR00000928324,2.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
74013,chr1,99738879,99739345,ENSR00000010583,NaN,2.0,NaN,NaN,NaN,NaN
74014,chr1,99745620,99746316,ENSR00000366709,NaN,2.0,NaN,NaN,NaN,NaN
74015,chr1,99772574,99773024,ENSR00000366717,NaN,2.0,NaN,NaN,NaN,NaN
74016,chr1,99773776,99774196,ENSR00000366718,NaN,2.0,NaN,NaN,NaN,NaN


In [8]:
regbuild_bed=pybedtools.BedTool.from_dataframe(regbuild)

## eQTL data

In [9]:
header=pd.read_csv('/nfs/research1/zerbino/jhidalgo/inteql_GTEX_v8/data/original-data/eQTL_catalogue/lead_pairs/split/header.tsv',sep='\t').columns.tolist()

In [10]:
chr1=pd.read_csv('/nfs/research1/zerbino/jhidalgo/inteql_GTEX_v8/data/original-data/eQTL_catalogue/lead_pairs/split/chr1.tsv',sep='\t',header=None,names=header)

In [11]:
chr1.sort_values('position')

,eqtl_id,Alasoo_2018.macrophage_IFNg.beta,Alasoo_2018.macrophage_IFNg+Salmonella.beta,Alasoo_2018.macrophage_naive.beta,Alasoo_2018.macrophage_Salmonella.beta,BLUEPRINT_PE.T-cell.beta,BLUEPRINT_SE.monocyte.beta,BLUEPRINT_SE.neutrophil.beta,BrainSeq.brain.beta,FUSION.adipose_naive.beta,...,Schwartzentruber_2018.sensory_neuron.se,TwinsUK.blood.se,TwinsUK.fat.se,TwinsUK.LCL.se,TwinsUK.skin.se,van_de_Bunt_2015.pancreatic_islet.se,molecular_trait_id,chromosome,position,variant
4524,chr1_54490_G_A.ENSG00000238009,NaN,NaN,NaN,NaN,NaN,NaN,0.092465,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,ENSG00000238009,1,54490,chr1_54490_G_A
4523,chr1_63697_T_C.ENSG00000238009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,ENSG00000238009,1,63697,chr1_63697_T_C
4526,chr1_64649_A_C.ENSG00000238009,NaN,NaN,NaN,NaN,NaN,NaN,0.275543,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,ENSG00000238009,1,64649,chr1_64649_A_C
4525,chr1_115746_C_T.ENSG00000238009,NaN,NaN,NaN,NaN,NaN,NaN,0.875146,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,ENSG00000238009,1,115746,chr1_115746_C_T
4596,chr1_115746_C_T.ENSG00000241860,NaN,NaN,NaN,NaN,NaN,0.430846,0.181977,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,ENSG00000241860,1,115746,chr1_115746_C_T
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4670,chr1_248733454_T_C.ENSG00000259823,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.601130,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,ENSG00000259823,1,248733454,chr1_248733454_T_C
4669,chr1_248739879_C_T.ENSG00000259823,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.499980,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,ENSG00000259823,1,248739879,chr1_248739879_C_T
5123,chr1_248739879_C_T.ENSG00000286015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.617971,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,ENSG00000286015,1,248739879,chr1_248739879_C_T
4671,chr1_248766065_C_T.ENSG00000259823,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.206758,NaN,...,NaN,0.122685,NaN,NaN,NaN,NaN,ENSG00000259823,1,248766065,chr1_248766065_C_T


In [12]:
chr1['chromosome']='chr'+chr1['chromosome'].astype(str)

# A (Variants)

In [287]:
variants=pd.DataFrame({'chromosome':chr1['chromosome'],'position_start':chr1['position'],'position_end':chr1['position'],'variant':chr1['variant']}).drop_duplicates()

In [288]:
variants

,chromosome,position_start,position_end,variant
0,chr1,169796199,169796199,chr1_169796199_G_A
1,chr1,169845687,169845687,chr1_169845687_C_CTTCCA
2,chr1,169821020,169821020,chr1_169821020_T_G
3,chr1,169982267,169982267,chr1_169982267_A_T
4,chr1,169787407,169787407,chr1_169787407_G_C
...,...,...,...,...
5140,chr1,233703526,233703526,chr1_233703526_T_C
5141,chr1,208262975,208262975,chr1_208262975_T_G
5142,chr1,208270332,208270332,chr1_208270332_C_A
5144,chr1,150320113,150320113,chr1_150320113_C_T


In [289]:
variants_bed=pybedtools.BedTool.from_dataframe(variants)

In [290]:
merged_var_data=variants

## eQTL - RegBuild intersection

In [291]:
eqtl_regbuild=variants_bed.intersect(regbuild_bed,wa=True,wb=True,loj=True).to_dataframe(names=['var_chr','var_start','var_end','variant','chr','start','end','reg_id']+regbuild.columns[-6:].to_list()).replace('.',np.nan)


/homes/jhidalgo/.local/lib/python3.8/subprocess.py:849: RuntimeWarning: line buffering (buffering=1) isn't supported in binary mode, the default buffer size will be used
  self.stderr = io.open(errread, 'rb', bufsize)


In [292]:
eqtl_regbuild[eqtl_regbuild.columns[-6:]]=eqtl_regbuild[eqtl_regbuild.columns[-6:]].astype('float')

In [293]:
eqtl_regbuild=eqtl_regbuild[eqtl_regbuild.columns[:4]].merge(eqtl_regbuild.groupby('variant',as_index=False).mean()).drop_duplicates(ignore_index=True)

In [294]:
eqtl_regbuild=eqtl_regbuild[eqtl_regbuild.columns[3:]]

In [295]:
new_names = [(i,'var_'+i) for i in eqtl_regbuild.columns[-8:].values]
eqtl_regbuild.rename(columns = dict(new_names), inplace=True)

In [296]:
eqtl_regbuild

,variant,var_start,var_end,var_CTCF_binding_site,var_TF_binding_site,var_enhancer,var_open_chromatin_region,var_promoter,var_promoter_flanking_region
0,chr1_169796199_G_A,169792002.0,169796999.0,NaN,NaN,NaN,NaN,4.0,NaN
1,chr1_169845687_C_CTTCCA,-1.0,-1.0,NaN,NaN,NaN,NaN,NaN,NaN
2,chr1_169821020_T_G,-1.0,-1.0,NaN,NaN,NaN,NaN,NaN,NaN
3,chr1_169982267_A_T,-1.0,-1.0,NaN,NaN,NaN,NaN,NaN,NaN
4,chr1_169787407_G_C,-1.0,-1.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
4818,chr1_233703526_T_C,-1.0,-1.0,NaN,NaN,NaN,NaN,NaN,NaN
4819,chr1_208262975_T_G,-1.0,-1.0,NaN,NaN,NaN,NaN,NaN,NaN
4820,chr1_208270332_C_A,208267601.0,208273000.0,NaN,NaN,NaN,NaN,NaN,1.0
4821,chr1_150320113_C_T,150320001.0,150320200.0,2.0,NaN,NaN,NaN,NaN,NaN


In [297]:
merged_var_data=merged_var_data.merge(eqtl_regbuild)

#### When not intersecting with an element, var_start - end = variant position

In [298]:
for index in merged_var_data[merged_var_data['var_start']==-1].index:
    merged_var_data['var_start'][index]=merged_var_data['position_start'][index]
    merged_var_data['var_end'][index]=merged_var_data['position_start'][index]

<ipython-input-298-cfb1189a6a54>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_var_data['var_start'][index]=merged_var_data['position_start'][index]
<ipython-input-298-cfb1189a6a54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_var_data['var_end'][index]=merged_var_data['position_start'][index]


In [299]:
merged_var_data

,chromosome,position_start,position_end,variant,var_start,var_end,var_CTCF_binding_site,var_TF_binding_site,var_enhancer,var_open_chromatin_region,var_promoter,var_promoter_flanking_region
0,chr1,169796199,169796199,chr1_169796199_G_A,169792002.0,169796999.0,NaN,NaN,NaN,NaN,4.0,NaN
1,chr1,169845687,169845687,chr1_169845687_C_CTTCCA,169845687.0,169845687.0,NaN,NaN,NaN,NaN,NaN,NaN
2,chr1,169821020,169821020,chr1_169821020_T_G,169821020.0,169821020.0,NaN,NaN,NaN,NaN,NaN,NaN
3,chr1,169982267,169982267,chr1_169982267_A_T,169982267.0,169982267.0,NaN,NaN,NaN,NaN,NaN,NaN
4,chr1,169787407,169787407,chr1_169787407_G_C,169787407.0,169787407.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
4818,chr1,233703526,233703526,chr1_233703526_T_C,233703526.0,233703526.0,NaN,NaN,NaN,NaN,NaN,NaN
4819,chr1,208262975,208262975,chr1_208262975_T_G,208262975.0,208262975.0,NaN,NaN,NaN,NaN,NaN,NaN
4820,chr1,208270332,208270332,chr1_208270332_C_A,208267601.0,208273000.0,NaN,NaN,NaN,NaN,NaN,1.0
4821,chr1,150320113,150320113,chr1_150320113_C_T,150320001.0,150320200.0,2.0,NaN,NaN,NaN,NaN,NaN


In [304]:
merged_var_data['var_start']=merged_var_data['var_start'].astype(int)
merged_var_data['var_end']=merged_var_data['var_end'].astype(int)

In [305]:
merged_var_bed=pybedtools.BedTool.from_dataframe(merged_var_data[['chromosome','var_start','var_end','variant']])

# TODO check effect of mean reg element start and end

# B (TSS)

## TSS

### Gencode v35

In [48]:
gencode=gffpd.read_gff3('/nfs/research1/zerbino/jhidalgo/inteql_GTEX_v8/data/original-data/gencode/gencode.v35.annotation.gff3').attributes_to_columns()

In [49]:
gencode=gencode[gencode['seq_id']=='chr1']

In [50]:
gencode=gencode[gencode['type']=='gene']

In [51]:
gencode['gene_id']=gencode['gene_id'].str.split('.',expand=True)[0]

In [496]:
gencode

,seq_id,source,type,start,end,score,strand,phase,attributes,ID,...,havana_transcript,hgnc_id,level,ont,protein_id,tag,transcript_id,transcript_name,transcript_support_level,transcript_type
0,chr1,HAVANA,gene,11869,14409,.,+,.,ID=ENSG00000223972.5;gene_id=ENSG00000223972.5...,ENSG00000223972.5,...,None,HGNC:37102,2,None,None,None,None,None,None,None
12,chr1,HAVANA,gene,14404,29570,.,-,.,ID=ENSG00000227232.5;gene_id=ENSG00000227232.5...,ENSG00000227232.5,...,None,HGNC:38034,2,None,None,None,None,None,None,None
25,chr1,ENSEMBL,gene,17369,17436,.,-,.,ID=ENSG00000278267.1;gene_id=ENSG00000278267.1...,ENSG00000278267.1,...,None,HGNC:50039,3,None,None,None,None,None,None,None
28,chr1,HAVANA,gene,29554,31109,.,+,.,ID=ENSG00000243485.5;gene_id=ENSG00000243485.5...,ENSG00000243485.5,...,None,HGNC:52482,2,None,None,ncRNA_host,None,None,None,None
36,chr1,ENSEMBL,gene,30366,30503,.,+,.,ID=ENSG00000284332.1;gene_id=ENSG00000284332.1...,ENSG00000284332.1,...,None,HGNC:35294,3,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271064,chr1,HAVANA,gene,248850006,248859144,.,-,.,ID=ENSG00000171163.16;gene_id=ENSG00000171163....,ENSG00000171163.16,...,None,HGNC:26049,1,None,None,None,None,None,None,None
271410,chr1,HAVANA,gene,248859164,248864796,.,+,.,ID=ENSG00000227237.1;gene_id=ENSG00000227237.1...,ENSG00000227237.1,...,None,None,1,None,None,None,None,None,None,None
271415,chr1,HAVANA,gene,248906196,248919946,.,+,.,ID=ENSG00000185220.12;gene_id=ENSG00000185220....,ENSG00000185220.12,...,None,HGNC:19399,2,None,None,None,None,None,None,None
271442,chr1,ENSEMBL,gene,248912690,248912795,.,-,.,ID=ENSG00000200495.1;gene_id=ENSG00000200495.1...,ENSG00000200495.1,...,None,HGNC:48168,3,None,None,None,None,None,None,None


In [53]:
gencode_simp=gencode[['gene_id','start','end','strand']]

In [54]:
gencode_simp=gencode_simp.set_index('gene_id')

### Simple approach to get tss from Gencode

In [78]:
TSS=pd.concat([gencode_simp[gencode_simp['strand']=='+']['start'],gencode_simp[gencode_simp['strand']=='-']['end']]).sort_values().reset_index(name='TSS')

In [104]:
TSS

,gene_id,TSS,chr
0,ENSG00000223972,11869,chr1
1,ENSG00000278267,17436,chr1
2,ENSG00000243485,29554,chr1
3,ENSG00000227232,29570,chr1
4,ENSG00000284332,30366,chr1
...,...,...,...
5471,ENSG00000171163,248859144,chr1
5472,ENSG00000227237,248859164,chr1
5473,ENSG00000185220,248906196,chr1
5474,ENSG00000200495,248912795,chr1


# TSS - eQTLs

In [131]:
TSS['chr']='chr1'

In [132]:
TSS_bed=pybedtools.BedTool.from_dataframe(TSS[['chr','TSS','TSS','gene_id']])

In [361]:
merged_TSS_data=TSS_bed.intersect(regbuild_bed,wa=True,wb=True,loj=True).to_dataframe(names=['TSS_chr','TSS_start','TSS_end','gene_id','chr','start','end','reg_id']+regbuild.columns[-6:].to_list()).replace('.',np.nan)

/homes/jhidalgo/.local/lib/python3.8/subprocess.py:849: RuntimeWarning: line buffering (buffering=1) isn't supported in binary mode, the default buffer size will be used
  self.stderr = io.open(errread, 'rb', bufsize)


In [363]:
merged_TSS_data[merged_TSS_data.columns[-6:]]=merged_TSS_data[merged_TSS_data.columns[-6:]].astype('float')

In [365]:
merged_TSS_data=merged_TSS_data[merged_TSS_data.columns[:4]].merge(merged_TSS_data.groupby('TSS_start',as_index=False).mean()).drop_duplicates(ignore_index=True)

In [369]:
merged_TSS_data=merged_TSS_data.drop(columns=['TSS_start']).rename(columns={'TSS_end':'TSS'})

In [370]:
new_names = [(i,'TSS_'+i) for i in merged_TSS_data.columns[-8:].values]
merged_TSS_data.rename(columns = dict(new_names), inplace=True)

#### When not intersecting with an element, TSS_start - end = TSS position

In [372]:
for index in merged_TSS_data[merged_TSS_data['TSS_start']==-1].index:
    merged_TSS_data['TSS_start'][index]=merged_TSS_data['TSS'][index]
    merged_TSS_data['TSS_end'][index]=merged_TSS_data['TSS'][index]

<ipython-input-372-b691c8e0a114>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_TSS_data['TSS_start'][index]=merged_TSS_data['TSS'][index]
<ipython-input-372-b691c8e0a114>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_TSS_data['TSS_end'][index]=merged_TSS_data['TSS'][index]


In [373]:
merged_TSS_data

,TSS_chr,TSS,gene_id,TSS_start,TSS_end,TSS_CTCF_binding_site,TSS_TF_binding_site,TSS_enhancer,TSS_open_chromatin_region,TSS_promoter,TSS_promoter_flanking_region
0,chr1,11869,ENSG00000223972,11869.0,11869.0,NaN,NaN,NaN,NaN,NaN,NaN
1,chr1,17436,ENSG00000278267,17436.0,17436.0,NaN,NaN,NaN,NaN,NaN,NaN
2,chr1,29554,ENSG00000243485,20602.0,33399.0,NaN,NaN,NaN,NaN,4.0,NaN
3,chr1,29570,ENSG00000227232,20602.0,33399.0,NaN,NaN,NaN,NaN,4.0,NaN
4,chr1,30366,ENSG00000284332,20602.0,33399.0,NaN,NaN,NaN,NaN,4.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
5471,chr1,248859144,ENSG00000171163,248856002.0,248862599.0,NaN,NaN,NaN,NaN,3.0,NaN
5472,chr1,248859164,ENSG00000227237,248856002.0,248862599.0,NaN,NaN,NaN,NaN,3.0,NaN
5473,chr1,248906196,ENSG00000185220,248904602.0,248908999.0,NaN,NaN,NaN,NaN,4.0,NaN
5474,chr1,248912795,ENSG00000200495,248912795.0,248912795.0,NaN,NaN,NaN,NaN,NaN,NaN


In [374]:
merged_TSS_data['TSS_start']=merged_TSS_data['TSS_start'].astype(int)
merged_TSS_data['TSS_end']=merged_TSS_data['TSS_end'].astype(int)

In [375]:
merged_TSS_bed=pybedtools.BedTool.from_dataframe(merged_TSS_data[['TSS_chr','TSS_start','TSS_end','gene_id']])

#  OLD SECTION

#### RegBuild intersection with eQTL TSS

In [142]:
merged_TSS_data.count()

TSS                             5476
gene_id                         5476
TSS_start                       5476
TSS_end                         5476
TSS_CTCF_binding_site            365
TSS_TF_binding_site               57
TSS_enhancer                     144
TSS_open_chromatin_region         51
TSS_promoter                    2623
TSS_promoter_flanking_region     506
dtype: int64

In [143]:
# eQTL_TSS_bed=pybedtools.BedTool.from_dataframe(pd.DataFrame({'chr':'chr1','start':merged_var_data['TSS'].fillna(0).astype(int),'end':merged_var_data['TSS'].fillna(0).astype(int),'eQTL':merged_var_data['variant']}))


In [144]:
# eQTL_TSS_bed.intersect(regbuild_bed,wa=True,wb=True,loj=True).to_dataframe().groupby('itemRgb').count()

In [147]:
# names='chrom	start	end	variant	score	TSS_element_start	TSS_element_end	TSS_element_ID	TSS_element_type	TSS_element_Activity'.split()


In [146]:
# eQTL_TSS_regbuild=eQTL_TSS_bed.intersect(regbuild_bed,wa=True,wb=True).to_dataframe(names=names)[['variant','TSS_element_start','TSS_element_end','TSS_element_ID','TSS_element_type','TSS_element_Activity']]


In [145]:
# merged_var_data.merge(eQTL_TSS_regbuild,how='left')

# DNASE

In [151]:
dnase=pd.read_csv('/nfs/research1/zerbino/jhidalgo/inteql_GTEX_v8/data/processed_data/monocyte-ML-input/dnase_seq.bed',sep='\t')

In [152]:
dnase=dnase[dnase['chr']=='chr1']

In [153]:
dnase

,chr,start,end,count
0,chr1,128619,128769,1
1,chr1,180756,180906,1
2,chr1,181395,181415,3
3,chr1,181415,181545,5
4,chr1,181545,181565,2
...,...,...,...,...
74294,chr1,248924802,248924945,4
74295,chr1,248924945,248924947,2
74296,chr1,248924947,248924952,1
74297,chr1,248925318,248925468,1


In [154]:
dnase_bed=pybedtools.BedTool.from_dataframe(dnase)

## DNAse-seq intersections

 - A (DNAse not intersected with TSS for Bio reasons)

In [309]:
dnase_variants=merged_var_bed.intersect(dnase_bed,wa=True,wb=True).to_dataframe(names=['chromosome','position_start','position_end','variant','var_dnase_chrom','var_dnase_start','var_dnase_end','var_dnase']).groupby('variant',as_index=False).mean()

/homes/jhidalgo/.local/lib/python3.8/subprocess.py:849: RuntimeWarning: line buffering (buffering=1) isn't supported in binary mode, the default buffer size will be used
  self.stderr = io.open(errread, 'rb', bufsize)


In [313]:
merged_var_data=merged_var_data.merge(dnase_variants[dnase_variants.columns[[0,5]]],how='left')

In [314]:
merged_var_data

,chromosome,position_start,position_end,variant,var_start,var_end,var_CTCF_binding_site,var_TF_binding_site,var_enhancer,var_open_chromatin_region,var_promoter,var_promoter_flanking_region,var_dnase
0,chr1,169796199,169796199,chr1_169796199_G_A,169792002,169796999,NaN,NaN,NaN,NaN,4.0,NaN,3.333333
1,chr1,169845687,169845687,chr1_169845687_C_CTTCCA,169845687,169845687,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,chr1,169821020,169821020,chr1_169821020_T_G,169821020,169821020,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,chr1,169982267,169982267,chr1_169982267_A_T,169982267,169982267,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,chr1,169787407,169787407,chr1_169787407_G_C,169787407,169787407,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4818,chr1,233703526,233703526,chr1_233703526_T_C,233703526,233703526,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4819,chr1,208262975,208262975,chr1_208262975_T_G,208262975,208262975,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4820,chr1,208270332,208270332,chr1_208270332_C_A,208267601,208273000,NaN,NaN,NaN,NaN,NaN,1.0,NaN
4821,chr1,150320113,150320113,chr1_150320113_C_T,150320001,150320200,2.0,NaN,NaN,NaN,NaN,NaN,NaN


# OLD SECTION

## CAGE

In [5]:
cage=pd.read_csv('/nfs/research1/zerbino/jhidalgo/inteql_GTEX_v8/data/processed_data/monocyte-ML-input/cage.bed',sep='\t')

In [6]:
cage=cage[cage['chrom']=='chr1']

#### RegBuild intersection with CAGE: Promoters selected

In [228]:
regbuild_cage=regbuild_bed.intersect(cage_top1000_bed,wa=True,wb=True,loj=True).to_dataframe()

/homes/jhidalgo/.local/lib/python3.8/subprocess.py:849: RuntimeWarning: line buffering (buffering=1) isn't supported in binary mode, the default buffer size will be used
  self.stderr = io.open(errread, 'rb', bufsize)


In [229]:
regbuild_cage

,chrom,start,end,name,score,strand,thickStart,thickEnd,itemRgb,blockCount
0,chr1,100034201,100035200,ENSR00000253249,CTCF_binding_site,ACTIVE,.,-1,-1,.
1,chr1,100127601,100127800,ENSR00000366775,CTCF_binding_site,INACTIVE,.,-1,-1,.
2,chr1,100130401,100130600,ENSR00000928323,CTCF_binding_site,INACTIVE,.,-1,-1,.
3,chr1,100135001,100135200,ENSR00000366777,CTCF_binding_site,INACTIVE,.,-1,-1,.
4,chr1,100147801,100148400,ENSR00000928324,CTCF_binding_site,INACTIVE,.,-1,-1,.
...,...,...,...,...,...,...,...,...,...,...
54528,chr1,99738879,99739345,ENSR00000010583,TF_binding_site,INACTIVE,.,-1,-1,.
54529,chr1,99745620,99746316,ENSR00000366709,TF_binding_site,INACTIVE,.,-1,-1,.
54530,chr1,99772574,99773024,ENSR00000366717,TF_binding_site,INACTIVE,.,-1,-1,.
54531,chr1,99773776,99774196,ENSR00000366718,TF_binding_site,INACTIVE,.,-1,-1,.


In [230]:
regbuild_cage.groupby('score').count()['chrom']

score
CTCF_binding_site           15857
TF_binding_site              2502
enhancer                    10857
open_chromatin_region        7997
promoter                     4353
promoter_flanking_region    12967
Name: chrom, dtype: int64

In [231]:
regbuild_cage[regbuild_cage['thickStart'] != '.'].groupby('score').count()['chrom']

score
CTCF_binding_site             47
TF_binding_site                1
enhancer                       4
open_chromatin_region          2
promoter                    1651
promoter_flanking_region      31
Name: chrom, dtype: int64

## CAGE-Gencode intersection check

In [20]:
TSS_bed=pybedtools.BedTool.from_dataframe(pd.DataFrame({'chr':'chr1','Start':TSS,'End':TSS}))

In [21]:
cage_bed=pybedtools.BedTool.from_dataframe(cage)

In [24]:
cage_tss=cage_bed.intersect(TSS_bed,wa=True,wb=True,loj=True).to_dataframe()

/homes/jhidalgo/.local/lib/python3.8/subprocess.py:849: RuntimeWarning: line buffering (buffering=1) isn't supported in binary mode, the default buffer size will be used
  self.stderr = io.open(errread, 'rb', bufsize)


#### All CAGE peaks values

In [35]:
cage_tss['name'].describe()

count     1813.000000
mean       832.031164
std       3636.205305
min          2.500000
25%         13.500000
50%         64.500000
75%        417.500000
max      71749.500000
Name: name, dtype: float64

#### CAGE score when intersecting with Gencode TSS

In [27]:
cage_tss[cage_tss['score']!='.']['name'].describe()

count      982.000000
mean      1348.031568
std       4662.877531
min          2.500000
25%         51.000000
50%        244.750000
75%        852.500000
max      71749.500000
Name: name, dtype: float64

#### CAGE score when not intersecting with Gencode TSS

In [28]:
cage_tss[cage_tss['score']=='.']['name'].describe()

count      831.000000
mean       222.268953
std       1574.834271
min          2.500000
25%          5.500000
50%         14.500000
75%         63.750000
max      37175.500000
Name: name, dtype: float64

# Top 1000 CAGE peaks

In [44]:
cage_top1000=cage.loc[cage['cage'].nlargest(1000).index]

In [93]:
cage_top1000_bed=pybedtools.BedTool.from_dataframe(cage_top1000)

In [45]:
cage_top1000

,chrom,start,end,cage
196,chr1,26280046,26280156,71749.5
200,chr1,26317933,26317974,38776.0
563,chr1,111473890,111473992,37175.5
738,chr1,154183153,154183208,31458.0
860,chr1,161215213,161215300,29923.0
...,...,...,...,...
220,chr1,27409921,27409954,5.5
460,chr1,76080114,76080161,5.5
494,chr1,93136710,93136757,5.5
589,chr1,114719580,114719710,5.5


# Histone Marks

In [223]:
H3K27ac=pd.read_csv('/nfs/research1/zerbino/jhidalgo/inteql_GTEX_v8/data/processed_data/monocyte-ML-input/H3K27ac-human.bed',sep='\t')

In [224]:
H3K4me3=pd.read_csv('/nfs/research1/zerbino/jhidalgo/inteql_GTEX_v8/data/processed_data/monocyte-ML-input/H3K4me3-human.bed',sep='\t')


In [225]:
H3K27ac

,chrom,start,end,H3K27ac-human_freq,H3K27ac-human_score
0,chr1,778337,778364,1,118.0
1,chr1,778364,778778,2,97.5
2,chr1,778778,778801,1,77.0
3,chr1,778893,779077,1,38.0
4,chr1,779077,779297,2,28.0
...,...,...,...,...,...
251898,chrY,21177632,21178034,1,35.0
251899,chrY,21214315,21214535,1,14.0
251900,chrY,21239955,21240494,1,21.0
251901,chrY,21240788,21241322,1,17.0


In [226]:
H3K4me3

,chrom,start,end,H3K4me3-human_freq,H3K4me3-human_score
0,chr1,778343,778362,1,54.000000
1,chr1,778362,778381,2,87.000000
2,chr1,778381,778710,3,89.333333
3,chr1,778710,778882,2,87.000000
4,chr1,778882,778952,3,79.666667
...,...,...,...,...,...
192735,chrY,56836353,56837136,1,34.000000
192736,chrY,56843141,56843432,1,18.000000
192737,chrY,56843872,56844132,1,12.000000
192738,chrY,56850060,56850320,1,12.000000


#### Sanity check with RegBuild

# OLD SECTION

In [395]:
pybedtools.BedTool.from_dataframe(H3K27ac).intersect(regbuild_bed,wa=True,wb=True).to_dataframe().groupby('blockCount').count()['chrom']

/homes/jhidalgo/.local/lib/python3.8/subprocess.py:849: RuntimeWarning: line buffering (buffering=1) isn't supported in binary mode, the default buffer size will be used
  self.stderr = io.open(errread, 'rb', bufsize)


blockCount
CTCF_binding_site            2528
TF_binding_site                99
enhancer                      410
open_chromatin_region         188
promoter                    27767
promoter_flanking_region     5302
Name: chrom, dtype: int64

In [396]:
pybedtools.BedTool.from_dataframe(H3K4me3).intersect(regbuild_bed,wa=True,wb=True).to_dataframe().groupby('blockCount').count()['chrom']

/homes/jhidalgo/.local/lib/python3.8/subprocess.py:849: RuntimeWarning: line buffering (buffering=1) isn't supported in binary mode, the default buffer size will be used
  self.stderr = io.open(errread, 'rb', bufsize)


blockCount
CTCF_binding_site            2517
TF_binding_site                 5
enhancer                       21
open_chromatin_region          14
promoter                    26423
promoter_flanking_region     1501
Name: chrom, dtype: int64

# Histon marks intersection

In [280]:
H3K27ac_bed=pybedtools.BedTool.from_dataframe(H3K27ac)

In [316]:
H3K4me3_bed=pybedtools.BedTool.from_dataframe(H3K4me3)

- A

In [ ]:
# H3K27ac

In [338]:
H3K27ac_var=merged_var_bed.intersect(H3K27ac_bed,wa=True,wb=True,loj=True).to_dataframe(names=['chr','start','end','variant','H_chr','H_start','H_end','var_H3K27ac_freq','var_H3K27ac_score']).replace('.',0)


/homes/jhidalgo/.local/lib/python3.8/subprocess.py:849: RuntimeWarning: line buffering (buffering=1) isn't supported in binary mode, the default buffer size will be used
  self.stderr = io.open(errread, 'rb', bufsize)


In [340]:
H3K27ac_var['var_H3K27ac_freq']=H3K27ac_var['var_H3K27ac_freq'].astype(int)

In [344]:
H3K27ac_var=H3K27ac_var.groupby('variant',as_index=False).mean()[['variant','var_H3K27ac_freq','var_H3K27ac_score']]

In [348]:
merged_var_data=merged_var_data.merge(H3K27ac_var,how='left')

In [ ]:
# H3K4me3

In [349]:
H3K4me3_var=merged_var_bed.intersect(H3K4me3_bed,wa=True,wb=True,loj=True).to_dataframe(names=['chr','start','end','variant','H_chr','H_start','H_end','var_H3K4me3_freq','var_H3K4me3_score']).replace('.',0)


/homes/jhidalgo/.local/lib/python3.8/subprocess.py:849: RuntimeWarning: line buffering (buffering=1) isn't supported in binary mode, the default buffer size will be used
  self.stderr = io.open(errread, 'rb', bufsize)


In [350]:
H3K4me3_var['var_H3K4me3_freq']=H3K4me3_var['var_H3K4me3_freq'].astype(int)

In [351]:
H3K4me3_var=H3K4me3_var.groupby('variant',as_index=False).mean()[['variant','var_H3K4me3_freq','var_H3K4me3_score']]

In [352]:
merged_var_data=merged_var_data.merge(H3K4me3_var,how='left')

In [353]:
merged_var_data

,chromosome,position_start,position_end,variant,var_start,var_end,var_CTCF_binding_site,var_TF_binding_site,var_enhancer,var_open_chromatin_region,var_promoter,var_promoter_flanking_region,var_dnase,var_H3K27ac_freq,var_H3K27ac_score,var_H3K4me3_freq,var_H3K4me3_score
0,chr1,169796199,169796199,chr1_169796199_G_A,169792002,169796999,NaN,NaN,NaN,NaN,4.0,NaN,3.333333,1.333333,157.5,1.8,463.066667
1,chr1,169845687,169845687,chr1_169845687_C_CTTCCA,169845687,169845687,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,-1.0,0.0,-1.000000
2,chr1,169821020,169821020,chr1_169821020_T_G,169821020,169821020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,-1.0,0.0,-1.000000
3,chr1,169982267,169982267,chr1_169982267_A_T,169982267,169982267,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,-1.0,0.0,-1.000000
4,chr1,169787407,169787407,chr1_169787407_G_C,169787407,169787407,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,-1.0,0.0,-1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4818,chr1,233703526,233703526,chr1_233703526_T_C,233703526,233703526,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,-1.0,0.0,-1.000000
4819,chr1,208262975,208262975,chr1_208262975_T_G,208262975,208262975,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,-1.0,0.0,-1.000000
4820,chr1,208270332,208270332,chr1_208270332_C_A,208267601,208273000,NaN,NaN,NaN,NaN,NaN,1.0,NaN,0.000000,-1.0,0.0,-1.000000
4821,chr1,150320113,150320113,chr1_150320113_C_T,150320001,150320200,2.0,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,-1.0,0.0,-1.000000


# HERE

- B

In [ ]:
# H3K27ac

In [381]:
H3K27ac_TSS=merged_TSS_bed.intersect(H3K27ac_bed,wa=True,wb=True,loj=True).to_dataframe(names=['chr','start','end','gene_id','H_chr','H_start','H_end','TSS_H3K27ac_freq','TSS_H3K27ac_score']).replace('.',0)


/homes/jhidalgo/.local/lib/python3.8/subprocess.py:849: RuntimeWarning: line buffering (buffering=1) isn't supported in binary mode, the default buffer size will be used
  self.stderr = io.open(errread, 'rb', bufsize)


In [382]:
H3K27ac_TSS['TSS_H3K27ac_freq']=H3K27ac_TSS['TSS_H3K27ac_freq'].astype(int)

In [383]:
H3K27ac_TSS=H3K27ac_TSS.groupby('gene_id',as_index=False).mean()[['gene_id','TSS_H3K27ac_freq','TSS_H3K27ac_score']]

In [384]:
merged_TSS_data=merged_TSS_data.merge(H3K27ac_TSS,how='left')

In [ ]:
# H3K4me3

In [385]:
H3K4me3_TSS=merged_TSS_bed.intersect(H3K4me3_bed,wa=True,wb=True,loj=True).to_dataframe(names=['chr','start','end','gene_id','H_chr','H_start','H_end','TSS_H3K4me3_freq','TSS_H3K4me3_score']).replace('.',0)


/homes/jhidalgo/.local/lib/python3.8/subprocess.py:849: RuntimeWarning: line buffering (buffering=1) isn't supported in binary mode, the default buffer size will be used
  self.stderr = io.open(errread, 'rb', bufsize)


In [386]:
H3K4me3_TSS['TSS_H3K4me3_freq']=H3K4me3_TSS['TSS_H3K4me3_freq'].astype(int)

In [387]:
H3K4me3_TSS=H3K4me3_TSS.groupby('gene_id',as_index=False).mean()[['gene_id','TSS_H3K4me3_freq','TSS_H3K4me3_score']]

In [388]:
merged_TSS_data=merged_TSS_data.merge(H3K4me3_TSS,how='left')

In [389]:
merged_TSS_data

,TSS_chr,TSS,gene_id,TSS_start,TSS_end,TSS_CTCF_binding_site,TSS_TF_binding_site,TSS_enhancer,TSS_open_chromatin_region,TSS_promoter,TSS_promoter_flanking_region,TSS_H3K27ac_freq,TSS_H3K27ac_score,TSS_H3K4me3_freq,TSS_H3K4me3_score
0,chr1,11869,ENSG00000223972,11869,11869,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,-1.000000,0.000000,-1.000000
1,chr1,17436,ENSG00000278267,17436,17436,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,-1.000000,0.000000,-1.000000
2,chr1,29554,ENSG00000243485,20602,33399,NaN,NaN,NaN,NaN,4.0,NaN,0.000000,-1.000000,0.000000,-1.000000
3,chr1,29570,ENSG00000227232,20602,33399,NaN,NaN,NaN,NaN,4.0,NaN,0.000000,-1.000000,0.000000,-1.000000
4,chr1,30366,ENSG00000284332,20602,33399,NaN,NaN,NaN,NaN,4.0,NaN,0.000000,-1.000000,0.000000,-1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5471,chr1,248859144,ENSG00000171163,248856002,248862599,NaN,NaN,NaN,NaN,3.0,NaN,1.400000,240.100000,1.916667,138.736111
5472,chr1,248859164,ENSG00000227237,248856002,248862599,NaN,NaN,NaN,NaN,3.0,NaN,1.400000,240.100000,1.916667,138.736111
5473,chr1,248906196,ENSG00000185220,248904602,248908999,NaN,NaN,NaN,NaN,4.0,NaN,1.285714,132.142857,1.800000,184.516667
5474,chr1,248912795,ENSG00000200495,248912795,248912795,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,-1.000000,0.000000,-1.000000


# Methylation

In [390]:
meth_CHG_file='/nfs/research1/zerbino/jhidalgo/inteql_GTEX_v8/data/original-data/monocyte_data/ENCODE/WGBS/CHG_values.bed'
meth_CHH_file='/nfs/research1/zerbino/jhidalgo/inteql_GTEX_v8/data/original-data/monocyte_data/ENCODE/WGBS/CHH_values.bed'
meth_CpG_file='/nfs/research1/zerbino/jhidalgo/inteql_GTEX_v8/data/original-data/monocyte_data/ENCODE/WGBS/CpG_values.bed'

In [391]:
CpG_bed=pybedtools.BedTool.from_dataframe(pd.read_csv(meth_CpG_file,sep='\t',header=None))
#names=['chr', 'start', 'end', 'M_CpG_freq', 'M_CpG_score']                                          

In [392]:
CHG_bed=pybedtools.BedTool.from_dataframe(pd.read_csv(meth_CHG_file,sep='\t',header=None))
#names=['chr', 'start', 'end', 'M_CHG_freq', 'M_CHG_score'])

In [393]:
CHH_bed=pybedtools.BedTool.from_dataframe(pd.read_csv(meth_CHH_file,sep='\t',header=None))
#names=['chr', 'start', 'end', 'M_CHH_freq', 'M_CHH_score'])

- A

In [ ]:
# CpG

In [431]:
CpG_var=merged_var_bed.intersect(CpG_bed,wa=True,wb=True).to_dataframe(names=['chr', 'start', 'end','variant','M_chr','M_start','M_end', 'var_M_CpG_freq', 'var_M_CpG_score'] )

/homes/jhidalgo/.local/lib/python3.8/subprocess.py:849: RuntimeWarning: line buffering (buffering=1) isn't supported in binary mode, the default buffer size will be used
  self.stderr = io.open(errread, 'rb', bufsize)


In [434]:
CpG_var_clean=CpG_var.groupby('variant',as_index=False).mean().iloc[:,[0,1,2,5,6]]

In [439]:
CpG_var_clean['var_M_CpG_density']=(CpG_var_clean['var_M_CpG_score']/(CpG_var_clean['end']-CpG_var_clean['start'])).replace(np.inf,0)

In [ ]:
# CHG

In [441]:
CHG_var=merged_var_bed.intersect(CHG_bed,wa=True,wb=True).to_dataframe(names=['chr', 'start', 'end','variant','M_chr','M_start','M_end', 'var_M_CHG_freq', 'var_M_CHG_score'] )

/homes/jhidalgo/.local/lib/python3.8/subprocess.py:849: RuntimeWarning: line buffering (buffering=1) isn't supported in binary mode, the default buffer size will be used
  self.stderr = io.open(errread, 'rb', bufsize)


In [442]:
CHG_var_clean=CHG_var.groupby('variant',as_index=False).mean().iloc[:,[0,1,2,5,6]]

In [443]:
CHG_var_clean['var_M_CHG_density']=(CHG_var_clean['var_M_CHG_score']/(CHG_var_clean['end']-CHG_var_clean['start'])).replace(np.inf,0)

In [ ]:
# CHH

In [444]:
CHH_var=merged_var_bed.intersect(CHH_bed,wa=True,wb=True).to_dataframe(names=['chr', 'start', 'end','variant','M_chr','M_start','M_end', 'var_M_CHH_freq', 'var_M_CHH_score'] )

/homes/jhidalgo/.local/lib/python3.8/subprocess.py:849: RuntimeWarning: line buffering (buffering=1) isn't supported in binary mode, the default buffer size will be used
  self.stderr = io.open(errread, 'rb', bufsize)


In [445]:
CHH_var_clean=CHH_var.groupby('variant',as_index=False).mean().iloc[:,[0,1,2,5,6]]

In [446]:
CHH_var_clean['var_M_CHH_density']=(CHH_var_clean['var_M_CHH_score']/(CHH_var_clean['end']-CHH_var_clean['start'])).replace(np.inf,0)

In [ ]:
# CpG,CHG,CHH

In [448]:
merged_var_data=merged_var_data.merge(CpG_var_clean.merge(CHG_var_clean,how='outer').merge(CHH_var_clean,how='outer'),how='left')

In [449]:
merged_var_data

,chromosome,position_start,position_end,variant,var_start,var_end,var_CTCF_binding_site,var_TF_binding_site,var_enhancer,var_open_chromatin_region,...,end,var_M_CpG_freq,var_M_CpG_score,var_M_CpG_density,var_M_CHG_freq,var_M_CHG_score,var_M_CHG_density,var_M_CHH_freq,var_M_CHH_score,var_M_CHH_density
0,chr1,169796199,169796199,chr1_169796199_G_A,169792002,169796999,NaN,NaN,NaN,NaN,...,169796999.0,22.093023,50.558140,0.010118,30.250000,4.850000,0.000971,20.774194,11.967742,0.002395
1,chr1,169845687,169845687,chr1_169845687_C_CTTCCA,169845687,169845687,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,chr1,169821020,169821020,chr1_169821020_T_G,169821020,169821020,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,chr1,169982267,169982267,chr1_169982267_A_T,169982267,169982267,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,chr1,169787407,169787407,chr1_169787407_G_C,169787407,169787407,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4818,chr1,233703526,233703526,chr1_233703526_T_C,233703526,233703526,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4819,chr1,208262975,208262975,chr1_208262975_T_G,208262975,208262975,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4820,chr1,208270332,208270332,chr1_208270332_C_A,208267601,208273000,NaN,NaN,NaN,NaN,...,208273000.0,21.646465,89.101010,0.016503,33.571429,7.742857,0.001434,26.058824,8.058824,0.001493
4821,chr1,150320113,150320113,chr1_150320113_C_T,150320001,150320200,2.0,NaN,NaN,NaN,...,150320200.0,5.000000,80.000000,0.402010,NaN,NaN,NaN,NaN,NaN,NaN


- B

In [ ]:
# CpG

In [450]:
CpG_TSS=merged_TSS_bed.intersect(CpG_bed,wa=True,wb=True).to_dataframe(names=['chr', 'start', 'end','gene_id','M_chr','M_start','M_end', 'TSS_M_CpG_freq', 'TSS_M_CpG_score'] )

/homes/jhidalgo/.local/lib/python3.8/subprocess.py:849: RuntimeWarning: line buffering (buffering=1) isn't supported in binary mode, the default buffer size will be used
  self.stderr = io.open(errread, 'rb', bufsize)


In [451]:
CpG_TSS_clean=CpG_TSS.groupby('gene_id',as_index=False).mean().iloc[:,[0,1,2,5,6]]

In [452]:
CpG_TSS_clean['TSS_M_CpG_density']=(CpG_TSS_clean['TSS_M_CpG_score']/(CpG_TSS_clean['end']-CpG_TSS_clean['start'])).replace(np.inf,0)

In [ ]:
# CHG

In [454]:
CHG_TSS=merged_TSS_bed.intersect(CHG_bed,wa=True,wb=True).to_dataframe(names=['chr', 'start', 'end','gene_id','M_chr','M_start','M_end', 'TSS_M_CHG_freq', 'TSS_M_CHG_score'] )

/homes/jhidalgo/.local/lib/python3.8/subprocess.py:849: RuntimeWarning: line buffering (buffering=1) isn't supported in binary mode, the default buffer size will be used
  self.stderr = io.open(errread, 'rb', bufsize)


In [455]:
CHG_TSS_clean=CHG_TSS.groupby('gene_id',as_index=False).mean().iloc[:,[0,1,2,5,6]]

In [456]:
CHG_TSS_clean['TSS_M_CHG_density']=(CHG_TSS_clean['TSS_M_CHG_score']/(CHG_TSS_clean['end']-CHG_TSS_clean['start'])).replace(np.inf,0)

In [ ]:
# CHH

In [457]:
CHH_TSS=merged_TSS_bed.intersect(CHH_bed,wa=True,wb=True).to_dataframe(names=['chr', 'start', 'end','gene_id','M_chr','M_start','M_end', 'TSS_M_CHH_freq', 'TSS_M_CHH_score'] )

/homes/jhidalgo/.local/lib/python3.8/subprocess.py:849: RuntimeWarning: line buffering (buffering=1) isn't supported in binary mode, the default buffer size will be used
  self.stderr = io.open(errread, 'rb', bufsize)


In [458]:
CHH_TSS_clean=CHH_TSS.groupby('gene_id',as_index=False).mean().iloc[:,[0,1,2,5,6]]

In [459]:
CHH_TSS_clean['TSS_M_CHH_density']=(CHH_TSS_clean['TSS_M_CHH_score']/(CHH_TSS_clean['end']-CHH_TSS_clean['start'])).replace(np.inf,0)

In [ ]:
# CpG,CHG,CHH

In [460]:
merged_TSS_data=merged_TSS_data.merge(CpG_TSS_clean.merge(CHG_TSS_clean,how='outer').merge(CHH_TSS_clean,how='outer'),how='left')

In [461]:
merged_TSS_data

,TSS_chr,TSS,gene_id,TSS_start,TSS_end,TSS_CTCF_binding_site,TSS_TF_binding_site,TSS_enhancer,TSS_open_chromatin_region,TSS_promoter,...,end,TSS_M_CpG_freq,TSS_M_CpG_score,TSS_M_CpG_density,TSS_M_CHG_freq,TSS_M_CHG_score,TSS_M_CHG_density,TSS_M_CHH_freq,TSS_M_CHH_score,TSS_M_CHH_density
0,chr1,11869,ENSG00000223972,11869,11869,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,chr1,17436,ENSG00000278267,17436,17436,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,chr1,29554,ENSG00000243485,20602,33399,NaN,NaN,NaN,NaN,4.0,...,33399.0,7.692308,68.846154,0.005380,26.000000,4.000000,0.000313,5.750000,36.000000,0.002813
3,chr1,29570,ENSG00000227232,20602,33399,NaN,NaN,NaN,NaN,4.0,...,33399.0,7.692308,68.846154,0.005380,26.000000,4.000000,0.000313,5.750000,36.000000,0.002813
4,chr1,30366,ENSG00000284332,20602,33399,NaN,NaN,NaN,NaN,4.0,...,33399.0,7.692308,68.846154,0.005380,26.000000,4.000000,0.000313,5.750000,36.000000,0.002813
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5471,chr1,248859144,ENSG00000171163,248856002,248862599,NaN,NaN,NaN,NaN,3.0,...,248862599.0,31.000000,58.647059,0.008890,46.448980,4.224490,0.000640,38.061728,7.012346,0.001063
5472,chr1,248859164,ENSG00000227237,248856002,248862599,NaN,NaN,NaN,NaN,3.0,...,248862599.0,31.000000,58.647059,0.008890,46.448980,4.224490,0.000640,38.061728,7.012346,0.001063
5473,chr1,248906196,ENSG00000185220,248904602,248908999,NaN,NaN,NaN,NaN,4.0,...,248908999.0,29.149254,61.223881,0.013924,33.884615,9.961538,0.002266,35.900000,9.700000,0.002206
5474,chr1,248912795,ENSG00000200495,248912795,248912795,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Merging datasets

In [555]:
chr1_merged=chr1.merge(merged_var_data,how='left').merge(merged_TSS_data,left_on='molecular_trait_id',right_on='gene_id',how='left')


In [557]:
chr1_merged

,eqtl_id,Alasoo_2018.macrophage_IFNg.beta,Alasoo_2018.macrophage_IFNg+Salmonella.beta,Alasoo_2018.macrophage_naive.beta,Alasoo_2018.macrophage_Salmonella.beta,BLUEPRINT_PE.T-cell.beta,BLUEPRINT_SE.monocyte.beta,BLUEPRINT_SE.neutrophil.beta,BrainSeq.brain.beta,FUSION.adipose_naive.beta,...,end_y,TSS_M_CpG_freq,TSS_M_CpG_score,TSS_M_CpG_density,TSS_M_CHG_freq,TSS_M_CHG_score,TSS_M_CHG_density,TSS_M_CHH_freq,TSS_M_CHH_score,TSS_M_CHH_density
0,chr1_169796199_G_A.ENSG00000000457,0.014878,0.019929,0.007145,0.008409,-0.009101,-0.003149,0.012835,-0.064450,-0.124903,...,169896999.0,23.849057,66.650943,0.008131,38.194444,4.916667,0.000600,27.355140,10.112150,0.001234
1,chr1_169845687_C_CTTCCA.ENSG00000000457,0.010908,-0.003541,0.007549,0.010561,-0.016851,-0.032392,0.011638,-0.059007,-0.112610,...,169896999.0,23.849057,66.650943,0.008131,38.194444,4.916667,0.000600,27.355140,10.112150,0.001234
2,chr1_169821020_T_G.ENSG00000000457,0.032490,0.035531,-0.010889,0.005276,-0.024435,-0.002244,0.015308,-0.017674,-0.078033,...,169896999.0,23.849057,66.650943,0.008131,38.194444,4.916667,0.000600,27.355140,10.112150,0.001234
3,chr1_169982267_A_T.ENSG00000000457,-0.015936,-0.023199,-0.003959,-0.011975,0.006480,0.006495,0.001639,0.040951,0.102934,...,169896999.0,23.849057,66.650943,0.008131,38.194444,4.916667,0.000600,27.355140,10.112150,0.001234
4,chr1_169787407_G_C.ENSG00000000457,0.017091,0.040628,0.002270,-0.012849,-0.046222,-0.001068,0.026307,-0.033465,-0.096427,...,169896999.0,23.849057,66.650943,0.008131,38.194444,4.916667,0.000600,27.355140,10.112150,0.001234
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5141,chr1_208262975_T_G.ENSG00000286198,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.169476,NaN,...,208273000.0,21.646465,89.101010,0.016503,33.571429,7.742857,0.001434,26.058824,8.058824,0.001493
5142,chr1_208270332_C_A.ENSG00000286198,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.481553,NaN,...,208273000.0,21.646465,89.101010,0.016503,33.571429,7.742857,0.001434,26.058824,8.058824,0.001493
5143,chr1_247323434_C_T.ENSG00000286216,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.039183,NaN,...,247349999.0,19.274510,53.637255,0.019177,27.526316,5.105263,0.001825,21.777778,10.688889,0.003822
5144,chr1_150320113_C_T.ENSG00000286219,-0.835799,-0.444700,-0.555345,-1.044910,-0.515010,-1.162060,-0.775167,NaN,NaN,...,149432199.0,14.787097,91.103226,0.002071,26.704918,10.065574,0.000229,20.393103,10.813793,0.000246


# OLD Section

##### RegBuild

In [407]:
# merged_chr1=chr1.merge(eqtl_regbuild_pivot.add_prefix('RegBuild_'),left_on='variant',right_index=True,how='left')

#### DNAse-seq

In [408]:
# merged_chr1=merged_chr1.merge(dnase_eqtl.rename(columns={'name':'DNAse-seq','thickEnd':'variant'})[['variant','DNAse-seq']],how='left')

#### Histone Marks

In [409]:
# merged_chr1=merged_chr1.merge(H3K27ac_eqtl[['name','thickEnd','itemRgb']].rename(columns={'name':'variant','thickEnd':'H3K27ac_freq','itemRgb':'H3K27ac_score'}),how='left')

In [410]:
# merged_chr1=merged_chr1.merge(H3K4me3_eqtl[['name','thickEnd','itemRgb']].rename(columns={'name':'variant','thickEnd':'H3K4me3_freq','itemRgb':'H3K4me3_score'}),how='left')

#### BS

In [411]:
# merged_chr1=merged_chr1.merge(CpG_eqtl,how='left')

In [412]:
# merged_chr1=merged_chr1.merge(CHG_eqtl,how='left')

In [413]:
# merged_chr1=merged_chr1.merge(CHH_eqtl,how='left')

#### HiC var-TSS

In [577]:
with open('/nfs/research1/zerbino/jhidalgo/inteql_GTEX_v8/data/original-data/HiC/averageHiC/chr1/Unique_positions.txt') as f:
    lines = f.read().splitlines()
for i in range(0, len(lines)): 
    lines[i] = int(lines[i])

In [715]:
hic_positions=pd.read_csv('/nfs/research1/zerbino/jhidalgo/inteql_GTEX_v8/data/original-data/HiC/averageHiC/chr1/Unique_positions.txt',header=None)

In [716]:
for index in chr1_merged.index:
    var_hic_pos=min(lines, key=lambda x:abs(x-chr1_merged['position'][index]))
    tss_hic_pos=min(lines, key=lambda x:abs(x-chr1_merged['TSS'][index]))
    if (abs(var_hic_pos-chr1_merged['position'][index])<5000) & (abs(tss_hic_pos-chr1_merged['TSS'][index])<5000):
        chr1_merged['var_hic_pos'][index]=var_hic_pos
        chr1_merged['tss_hic_pos'][index]=tss_hic_pos
    else:
        chr1_merged['var_hic_pos'][index]=0
        chr1_merged['tss_hic_pos'][index]=0

<ipython-input-716-523390f2179f>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chr1_merged['var_hic_pos'][index]=var_hic_pos
<ipython-input-716-523390f2179f>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chr1_merged['tss_hic_pos'][index]=tss_hic_pos
<ipython-input-716-523390f2179f>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chr1_merged['var_hic_pos'][index]=0
<ipython-input-716-523390f2179f>:9: SettingWithCopyWarning: 
A va

In [717]:
chr1_merged

,eqtl_id,Alasoo_2018.macrophage_IFNg.beta,Alasoo_2018.macrophage_IFNg+Salmonella.beta,Alasoo_2018.macrophage_naive.beta,Alasoo_2018.macrophage_Salmonella.beta,BLUEPRINT_PE.T-cell.beta,BLUEPRINT_SE.monocyte.beta,BLUEPRINT_SE.neutrophil.beta,BrainSeq.brain.beta,FUSION.adipose_naive.beta,...,TSS_M_CpG_density,TSS_M_CHG_freq,TSS_M_CHG_score,TSS_M_CHG_density,TSS_M_CHH_freq,TSS_M_CHH_score,TSS_M_CHH_density,var_hic_pos,tss_hic_pos,HiC_Contact
0,chr1_169796199_G_A.ENSG00000000457,0.014878,0.019929,0.007145,0.008409,-0.009101,-0.003149,0.012835,-0.064450,-0.124903,...,0.008131,38.194444,4.916667,0.000600,27.355140,10.112150,0.001234,169795860,169895859,0.0
1,chr1_169845687_C_CTTCCA.ENSG00000000457,0.010908,-0.003541,0.007549,0.010561,-0.016851,-0.032392,0.011638,-0.059007,-0.112610,...,0.008131,38.194444,4.916667,0.000600,27.355140,10.112150,0.001234,169845859,169895859,0.0
2,chr1_169821020_T_G.ENSG00000000457,0.032490,0.035531,-0.010889,0.005276,-0.024435,-0.002244,0.015308,-0.017674,-0.078033,...,0.008131,38.194444,4.916667,0.000600,27.355140,10.112150,0.001234,169820860,169895859,0.0
3,chr1_169982267_A_T.ENSG00000000457,-0.015936,-0.023199,-0.003959,-0.011975,0.006480,0.006495,0.001639,0.040951,0.102934,...,0.008131,38.194444,4.916667,0.000600,27.355140,10.112150,0.001234,169980860,169895859,0.0
4,chr1_169787407_G_C.ENSG00000000457,0.017091,0.040628,0.002270,-0.012849,-0.046222,-0.001068,0.026307,-0.033465,-0.096427,...,0.008131,38.194444,4.916667,0.000600,27.355140,10.112150,0.001234,169785860,169895859,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5141,chr1_208262975_T_G.ENSG00000286198,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.169476,NaN,...,0.016503,33.571429,7.742857,0.001434,26.058824,8.058824,0.001493,208261656,208271655,0.0
5142,chr1_208270332_C_A.ENSG00000286198,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.481553,NaN,...,0.016503,33.571429,7.742857,0.001434,26.058824,8.058824,0.001493,208271655,208271655,0.0
5143,chr1_247323434_C_T.ENSG00000286216,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.039183,NaN,...,0.019177,27.526316,5.105263,0.001825,21.777778,10.688889,0.003822,247321699,247346699,0.0
5144,chr1_150320113_C_T.ENSG00000286219,-0.835799,-0.444700,-0.555345,-1.044910,-0.515010,-1.162060,-0.775167,NaN,NaN,...,0.002071,26.704918,10.065574,0.000229,20.393103,10.813793,0.000246,0,0,0.0


In [718]:
chr1_hic_pairs=list(zip(chr1_merged['var_hic_pos'],chr1_merged['tss_hic_pos']))

In [694]:
test3=chr1_merged
test3['HiC_Contact']=0

In [719]:
# pd.concat([test3['var_hic_pos'],test3['tss_hic_pos']])
with open('/nfs/research1/zerbino/jhidalgo/inteql_GTEX_v8/data/original-data/HiC/averageHiC/chr1/required_pos.txt', 'w') as f:
    for item in sorted(set(list(set(test3['var_hic_pos'].unique()))+list(set(test3['tss_hic_pos'].unique())))):
        f.write("%s\n" % item)
# .to_csv('')

In [736]:
for chunk in pd.read_csv('/nfs/research1/zerbino/jhidalgo/inteql_GTEX_v8/data/original-data/HiC/averageHiC/chr1/filtered_hic',sep=' ',names=['A','chr','B','HiC_Contact'],chunksize=10000):
    for index in chunk.index:
        pair=chunk['A'][index],chunk['B'][index]
        if pair in chr1_hic_pairs:
            print(pair)
            test3.loc[test3[(test3['var_hic_pos']==pair[0]) & (test3['tss_hic_pos']==pair[1])].index,'HiC_Contact']=chunk['HiC_Contact'][index]
#         chr1_merged[(chr1_merged['var_hic_pos']==pair[0]) & (chr1_merged['tss_hic_pos']==pair[1])]['HiC_Contact']=test['HiC_Contact'][index]
# for pair in list(zip(test['A'],test['B'])):
#         print(pair)
#         169820860, 169895859

(10044942, 10209943)
(100894444, 100894445)
(101179444, 101234445)
(101849444, 101994445)
(10199942, 10209943)
(102314444, 102389445)
(10259942, 10384943)
(10259942, 10394944)
(10319942, 10384943)
(10319942, 10394944)
(107917378, 107962379)
(109007378, 109247379)
(109217378, 109247379)
(109242378, 109247379)
(109277378, 109282379)
(109277378, 109397379)
(109282378, 109282379)
(109432378, 109482379)
(109527378, 109687379)
(10959943, 10979944)
(109632378, 109667379)
(109632378, 109687379)
(109657378, 109667379)
(109657378, 109687379)
(109672378, 109687379)
(109677378, 109692379)
(109687378, 109687379)
(109687378, 109692379)
(1099620, 1114621)
(110057378, 110057379)
(110307378, 110337379)
(110382378, 110407379)
(110397378, 110407379)
(110887378, 110942379)
(110897378, 110962379)
(111192378, 111202379)
(111202378, 111202379)
(111447378, 111472379)
(111502378, 111562379)
(111922378, 111987379)
(112422378, 112707379)
(112522378, 112617379)
(112707378, 112707379)
(113062378, 113072379)
(11357

(247966698, 248046699)
(24808509, 25543510)
(248810801, 248825802)
(2493561, 2493562)
(25353509, 25428510)
(26168509, 26168510)
(26183509, 26188510)
(26203509, 26233510)
(26208509, 26233510)
(26323509, 26358510)
(26328509, 26358510)
(26353509, 26358510)
(26358509, 26358510)
(27573489, 27633490)
(27898489, 27913490)
(27903489, 27933490)
(27968489, 28088490)
(28158489, 28503489)
(28308489, 28503489)
(29088488, 29123489)
(29118488, 29228489)
(29198488, 29228489)
(29203488, 29228489)
(30517153, 31407154)
(31217153, 31407154)
(31392153, 31407154)
(31609399, 31644400)
(31759399, 32649400)
(31849399, 31944400)
(32634399, 32649400)
(32674399, 32739400)
(32709399, 32739400)
(32724399, 32739400)
(32739399, 32739400)
(3298436, 3308437)
(32994399, 33079400)
(33314399, 33349400)
(33339399, 33349400)
(33339399, 33429400)
(34849399, 34849400)
(34849399, 34859400)
(3618436, 3623437)
(37449399, 37474400)
(37499399, 37514400)
(37514399, 37514400)
(37584399, 37594400)
(37729328, 37764329)
(37739328, 3776

In [741]:
test3[test3['HiC_Contact']!=0]['HiC_Contact'].describe()

count    585.000000
mean       0.029027
std        0.046604
min        0.000044
25%        0.003430
50%        0.007784
75%        0.023231
max        0.262740
Name: HiC_Contact, dtype: float64

In [745]:
test3['NAs']=test3.isna().sum(axis=1)#.groupby(['Mt'])['count'].transform(max) == df['count']

In [755]:
test3.groupby(['molecular_trait_id'],sort=False,as_index=False)['NAs'].min()

,molecular_trait_id,NAs
0,ENSG00000000457,10
1,ENSG00000000460,10
2,ENSG00000000938,30
3,ENSG00000001460,41
4,ENSG00000001461,19
...,...,...
1940,ENSG00000286106,126
1941,ENSG00000286109,266
1942,ENSG00000286198,263
1943,ENSG00000286216,101


In [758]:
test3['HiC_distance']=abs(test3['var_hic_pos']-test3['tss_hic_pos'])

In [763]:
test3[test3['HiC_Contact']>0]

,eqtl_id,Alasoo_2018.macrophage_IFNg.beta,Alasoo_2018.macrophage_IFNg+Salmonella.beta,Alasoo_2018.macrophage_naive.beta,Alasoo_2018.macrophage_Salmonella.beta,BLUEPRINT_PE.T-cell.beta,BLUEPRINT_SE.monocyte.beta,BLUEPRINT_SE.neutrophil.beta,BrainSeq.brain.beta,FUSION.adipose_naive.beta,...,TSS_M_CHG_score,TSS_M_CHG_density,TSS_M_CHH_freq,TSS_M_CHH_score,TSS_M_CHH_density,var_hic_pos,tss_hic_pos,HiC_Contact,NAs,HiC_distance
6,chr1_169629757_G_C.ENSG00000000460,0.036219,0.056579,-0.036440,0.009718,-0.033352,-0.065001,-0.029941,-0.034172,0.027336,...,3.000000,0.001431,38.492958,5.154930,0.002458,169630762,169660763,0.007751,11,30001
11,chr1_27573355_T_TGCCA.ENSG00000000938,1.316990,0.359478,1.001060,0.420240,NaN,NaN,NaN,0.098242,0.353696,...,4.367965,0.000090,36.330380,7.111392,0.000147,27573489,27633490,0.002887,30,60001
22,chr1_32993202_A_G.ENSG00000004455,-0.023348,-0.016181,0.054461,0.011161,-0.012826,-0.035521,-0.074424,-0.017013,-0.039564,...,2.850000,0.001706,41.354167,3.750000,0.002244,32994399,33079400,0.004198,9,85001
24,chr1_54783481_T_C.ENSG00000006555,NaN,NaN,NaN,NaN,0.016904,NaN,0.054719,-0.025372,0.062545,...,3.254545,0.000880,43.848101,4.974684,0.001345,54784327,54799328,0.009796,112,15001
40,chr1_1668108_C_T.ENSG00000008128,NaN,NaN,NaN,NaN,0.099361,0.117853,0.038317,NaN,-0.161910,...,5.404762,0.000932,32.813333,8.053333,0.001389,1668561,1723562,0.005600,59,55001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5084,chr1_10318819_C_T.ENSG00000284735,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.550245,NaN,...,6.205479,0.000675,27.181208,6.872483,0.000747,10319942,10384943,0.002432,104,65001
5085,chr1_10258991_G_A.ENSG00000284735,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.033415,NaN,...,6.205479,0.000675,27.181208,6.872483,0.000747,10259942,10384943,0.001900,80,125001
5112,chr1_184559445_G_A.ENSG00000285847,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.084035,NaN,...,7.750000,0.004850,25.428571,13.642857,0.008537,184560866,184670867,0.002766,146,110001
5119,chr1_39748703_G_C.ENSG00000285905,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,39749328,39779329,0.009013,275,30001


In [762]:
test3.to_csv('/nfs/research1/zerbino/jhidalgo/inteql_GTEX_v8/data/processed_data/monocyte-ML-input/merged_datasets.csv',index=False)

# Pending

In [417]:
merged_chr1=merged_chr1.groupby('eqtl_id').mean()

In [418]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print((merged_chr1.isna().sum()/len(merged_chr1))*100)

Alasoo_2018.macrophage_IFNg.beta                  44.228527
Alasoo_2018.macrophage_IFNg+Salmonella.beta       46.230082
Alasoo_2018.macrophage_naive.beta                 40.788962
Alasoo_2018.macrophage_Salmonella.beta            43.023708
BLUEPRINT_PE.T-cell.beta                          39.914497
BLUEPRINT_SE.monocyte.beta                        40.963855
BLUEPRINT_SE.neutrophil.beta                      52.312476
BrainSeq.brain.beta                               21.162068
FUSION.adipose_naive.beta                         27.866304
FUSION.muscle_naive.beta                          36.494365
GENCORD.fibroblast.beta                           35.503304
GENCORD.LCL.beta                                  34.356782
GENCORD.T-cell.beta                               28.118927
GEUVADIS.LCL.beta                                 31.713953
GTEx.adipose_subcutaneous.beta                    18.577536
GTEx.adipose_visceral.beta                        17.975126
GTEx.adrenal_gland.beta                 

In [419]:
merged_chr1.to_csv('/nfs/research1/zerbino/jhidalgo/inteql_GTEX_v8/data/processed_data/monocyte-ML-input/merged_datasets_noHiC.csv',index=False)


# Genes

In [421]:
chr1[['molecular_trait_id','variant']]

,molecular_trait_id,variant
0,ENSG00000000457,chr1_169796199_G_A
1,ENSG00000000457,chr1_169845687_C_CTTCCA
2,ENSG00000000457,chr1_169821020_T_G
3,ENSG00000000457,chr1_169982267_A_T
4,ENSG00000000457,chr1_169787407_G_C
...,...,...
5141,ENSG00000286198,chr1_208262975_T_G
5142,ENSG00000286198,chr1_208270332_C_A
5143,ENSG00000286216,chr1_247323434_C_T
5144,ENSG00000286219,chr1_150320113_C_T


In [426]:
gencode[['gene_id',3,4,6]]

,gene_id,3,4,6
1,ENSG00000223972,11869,14409,+
5,ENSG00000223972,12010,13670,+
13,ENSG00000227232,14404,29570,-
26,ENSG00000278267,17369,17436,-
29,ENSG00000243485,29554,31097,+
...,...,...,...,...
234914,ENSG00000185220,248906196,248919946,+
234925,ENSG00000185220,248906243,248918573,+
234936,ENSG00000185220,248906372,248917401,+
234941,ENSG00000200495,248912690,248912795,-
